<a href="https://colab.research.google.com/github/harshada1442/Basic_Data_Science_Projects/blob/main/Main_food_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#                Data Processing

> We will resize all images in the input folder to the specified dimensions (in this case, 224x224 pixels) and save them in the output folder.



In [ ]:
def resize_images(input_folder, output_folder, target_width=224, target_height=224):
    # Iterate through all subdirectories (assuming each subdirectory is a category of food)
    for category in os.listdir(input_folder):
        category_input_folder = os.path.join(input_folder, category)
        category_output_folder = os.path.join(output_folder, category)
        os.makedirs(category_output_folder, exist_ok=True)

        # Iterate through all files in the current category folder
        for filename in os.listdir(category_input_folder):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                # Read the image
                img_path = os.path.join(category_input_folder, filename)
                img = cv2.imread(img_path)

                # Resize the image to the target dimensions
                resized_img = cv2.resize(img, (target_width, target_height))

                # Define the output path for the resized image dynamically
                output_path = os.path.join(category_output_folder, filename)

                # Save the resized image
                cv2.imwrite(output_path, resized_img)
                print(f'Resized and saved: {output_path}')

# Define the paths to the folder containing your food images and the output folder
input_folder = '/content/drive/MyDrive/Food Recognization/Images'
output_folder = '/content/drive/MyDrive/Food Recognization/resized_images/'

# Call the function to resize images in the input folder and save them to the output folder
resize_images(input_folder, output_folder)

print('Resizing completed.')


**Normalize the data**
we will Normalize the pixel values of the all images to a common scale, usually between 0 and 1.

In [ ]:
def normalize_images(input_folder, output_folder, target_width=224, target_height=224):
    # Iterate through all subfolders (categories) in the input folder
    for category in os.listdir(input_folder):
        category_input_folder = os.path.join(input_folder, category)
        category_output_folder = os.path.join(output_folder, category)
        os.makedirs(category_output_folder, exist_ok=True)

        # Iterate through all files in the current category folder
        for filename in os.listdir(category_input_folder):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                # Read the image
                img_path = os.path.join(category_input_folder, filename)
                img = cv2.imread(img_path)

                # Resize the image to the target dimensions
                resized_img = cv2.resize(img, (target_width, target_height))

                # Normalize pixel values to the range [0, 1]
                normalized_img = resized_img.astype(np.float32) / 255.0

                # Define the output path for the normalized image
                output_path = os.path.join(category_output_folder, filename)

                # Save the normalized image
                cv2.imwrite(output_path, (normalized_img * 255).astype(np.uint8))
                print(f'Normalized and saved: {output_path}')

input_main_folder = '/content/drive/MyDrive/Food Recognization/Images'
output_main_folder = '/content/drive/MyDrive/Food Recognization/resized_images/'

# Call the function to normalize images in the input folder and save them to the output folder
normalize_images(input_main_folder, output_main_folder)

print('Normalization completed.')



# Split the dataset
In this section we will Divide our dataset into three parts: training, validation, and testing sets. A common split is 70% for training, 15% for validation, and 15% for testing.

In [ ]:
# Define the path to your dataset directory on Google Drive
dataset_dir = '/content/drive/MyDrive/Food Recognization/resized_images'

# Define the paths for the training and testing datasets on Google Drive
train_dir = '/content/drive/MyDrive/Food Recognization/Split_data/training_set/'
test_dir = '/content/drive/MyDrive/Food Recognization/Split_data/testing_set/'

# Create output directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define the percentage split for training
train_percent = 0.7  # 70% for training

# Iterate through the subdirectories in the dataset directory (assuming each subdirectory represents a class)
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)

    # List all images in the class directory
    class_images = os.listdir(class_dir)

    # Calculate the number of images for training based on the desired train_percent
    num_train_images = int(len(class_images) * train_percent)

    # Ensure at least one image is included in the training set
    num_train_images = max(1, num_train_images)

    # Select the first num_train_images for training and the rest for testing
    train_images = class_images[:num_train_images]
    test_images = class_images[num_train_images:]

    # Move images to the training directory
    for image in train_images:
        src_path = os.path.join(class_dir, image)
        dst_path = os.path.join(train_dir, class_name, image)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy(src_path, dst_path)

    # Move images to the testing directory
    for image in test_images:
        src_path = os.path.join(class_dir, image)
        dst_path = os.path.join(test_dir, class_name, image)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy(src_path, dst_path)

print("Dataset split completed.")


Dataset split completed.


# Deep Learning Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
training_set = train_data_generator.flow_from_directory('/content/drive/MyDrive/Food Recognization/Split_data/training_set', target_size=(64, 64), batch_size=32, class_mode='categorical')


Found 421 images belonging to 22 classes.


In [ ]:
food_categories=["vada_pav", "uttapa", "upit", "sheera", "samosa", "sabudana_vada", "sabudana_khichdi", "poha", "pav_bhaji", "pani_puri", "Onion_pakoda", "misal_pav", "medu_vada", "masala_dosa", "kacchi_dabeli", "jalebi", "Idali_sambhar_chutany", "chole_bhature","Bhel_puri", "batata_vada","aloo_puri", "aloo_tikii"]
len(food_categories)

22

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=len(food_categories), activation='softmax'))  # Adjust len(food_categories) based on the number of food categories you have

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(training_set, epochs=25)  # You can adjust the number of epochs based on the performance on your validation set


Epoch 1/25
14/14 [==============================] - 137s 10s/step - loss: 3.1053 - accuracy: 0.0736
Epoch 2/25
14/14 [==============================] - 3s 212ms/step - loss: 3.0039 - accuracy: 0.1164
Epoch 3/25
14/14 [==============================] - 4s 240ms/step - loss: 2.9396 - accuracy: 0.1330
Epoch 4/25
14/14 [==============================] - 4s 305ms/step - loss: 2.8986 - accuracy: 0.1639
Epoch 5/25
14/14 [==============================] - 4s 272ms/step - loss: 2.8683 - accuracy: 0.1568
Epoch 6/25
14/14 [==============================] - 4s 295ms/step - loss: 2.8532 - accuracy: 0.1496
Epoch 7/25
14/14 [==============================] - 3s 219ms/step - loss: 2.8181 - accuracy: 0.1591
Epoch 8/25
14/14 [==============================] - 3s 223ms/step - loss: 2.7805 - accuracy: 0.1829
Epoch 9/25
14/14 [==============================] - 4s 233ms/step - loss: 2.6919 - accuracy: 0.1948
Epoch 10/25
14/14 [==============================] - 3s 212ms/step - loss: 2.7160 - accuracy: 0.1734

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the number of classes (food categories) in your dataset
num_classes = 22  # Replace 10 with the actual number of food categories in your dataset

# Define your CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # num_classes is the number of food categories

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(training_set, epochs=25)

Epoch 1/25
14/14 [==============================] - 81s 6s/step - loss: 3.1523 - accuracy: 0.0451
Epoch 2/25
14/14 [==============================] - 4s 271ms/step - loss: 3.0174 - accuracy: 0.1164
Epoch 3/25
14/14 [==============================] - 5s 342ms/step - loss: 2.9568 - accuracy: 0.1449
Epoch 4/25
14/14 [==============================] - 3s 232ms/step - loss: 2.9207 - accuracy: 0.1473
Epoch 5/25
14/14 [==============================] - 3s 226ms/step - loss: 2.8603 - accuracy: 0.1568
Epoch 6/25
14/14 [==============================] - 4s 308ms/step - loss: 2.7605 - accuracy: 0.1615
Epoch 7/25
14/14 [==============================] - 4s 288ms/step - loss: 2.6486 - accuracy: 0.1971
Epoch 8/25
14/14 [==============================] - 4s 249ms/step - loss: 2.5440 - accuracy: 0.2162
Epoch 9/25
14/14 [==============================] - 5s 326ms/step - loss: 2.4894 - accuracy: 0.2447
Epoch 10/25
14/14 [==============================] - 5s 300ms/step - loss: 2.3040 - accuracy: 0.3183
E

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the number of classes (food categories) in your dataset
num_classes = 22  # Replace 10 with the actual number of food categories in your dataset

# Define the image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define the path to your training data directory
train_data_dir = '/content/drive/MyDrive/Food Recognization/Split_data/training_set'

# Create ImageDataGenerator for training data with aggressive data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Define your CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add dropout with a rate of 0.5
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.0001)  # Experiment with different learning rates
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 50  # Increase the number of epochs for training
model.fit(train_generator, epochs=epochs)

# Save the trained model for future use
model.save('food_classification_model.h5')


Found 421 images belonging to 22 classes.


Epoch 1/50
14/14 [==============================] - 60s 4s/step - loss: 3.7348 - accuracy: 0.0736
Epoch 2/50
14/14 [==============================] - 66s 5s/step - loss: 3.0678 - accuracy: 0.1093
Epoch 3/50
14/14 [==============================] - 54s 4s/step - loss: 3.0085 - accuracy: 0.1188
Epoch 4/50
14/14 [==============================] - 54s 4s/step - loss: 3.0266 - accuracy: 0.1188
Epoch 5/50
14/14 [==============================] - 53s 4s/step - loss: 2.9676 - accuracy: 0.1116
Epoch 6/50
14/14 [==============================] - 53s 4s/step - loss: 2.9445 - accuracy: 0.1164
Epoch 7/50
14/14 [==============================] - 53s 4s/step - loss: 2.9445 - accuracy: 0.1306
Epoch 8/50
14/14 [==============================] - 53s 4s/step - loss: 2.9581 - accuracy: 0.1211
Epoch 9/50
14/14 [==============================] - 52s 4s/step - loss: 2.8962 - accuracy: 0.1283
Epoch 10/50
14/14 [==============================] - 62s 4s/step - loss: 2.8593 - accuracy: 0.1496
Epoch 11/50
14/14 [